In [1]:
import pandas as pd 
import inventorize as inv 
import numpy as np 
import os 
path = os.getcwd()
print(path)

/Users/dang/Desktop/Learning/Mygit/datascience_supplychain_practice/Jupyternotebook_source/Section_18_seasonal_inventory


In [2]:
retail = pd.read_csv('{0}/twentyeleven.csv'.format(path))

In [3]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484330 entries, 0 to 484329
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      484329 non-null  float64
 1   StockCode    484330 non-null  object 
 2   Description  484330 non-null  object 
 3   Quantity     484330 non-null  int64  
 4   InvoiceDate  484330 non-null  object 
 5   Price        484330 non-null  float64
 6   Customer ID  367058 non-null  float64
 7   Country      484330 non-null  object 
 8   date         484330 non-null  object 
 9   revenue      484330 non-null  float64
 10  year         484330 non-null  int64  
dtypes: float64(4), int64(2), object(5)
memory usage: 40.6+ MB


In [4]:
retail['InvoiceDate']  = pd.to_datetime(retail['InvoiceDate'])

In [5]:
retail['date'] = retail['InvoiceDate'].dt.strftime("%Y-%m-%d")

In [6]:
retail['date'] = pd.to_datetime(retail['date'])

In [9]:
retail['year'] = retail['date'].dt.year

In [10]:
total = retail.groupby(['Description']).agg(
    total_sales=('Quantity', 'sum'),
    price = ('Price', 'mean')
).reset_index()

In [11]:
total['sd'] = total['total_sales']*0.1

In [12]:
total['cost'] = total['price']*0.4

In [13]:
empty_data = pd.DataFrame()

In [15]:
for i in range(total.shape[0]):
    a = inv.MPN_singleperiod(
        total.loc[i,'total_sales'],
        total.loc[i,'sd'],
        total.loc[i, 'price'],
        total.loc[i, 'cost'], 0 , 0
    )
    b = pd.DataFrame(a, index=[0])
    b['decription'] = total.loc[i, 'Description']
    empty_data = pd.concat([empty_data, b], axis=0)


/Users/dang/Desktop/Learning/Mygit/datascience_supplychain_practice/env/lib/python3.7/site-packages/inventorize/__init__.py:696: RuntimeWarning: invalid value encountered in double_scalars
  quantity= scipy.stats.norm.ppf((p-c+b)/(p-c+b+c-g),mean,standerddeviation)


In [16]:
empty_data

,quantity,demand,sd,unitshort,shortagecost,cost,revenue,profit,soldatfullprice,sold_discount,decription
0,6132.526902,5981,598.1,170.460708,177.653908,2556.524336,6233.389716,3499.211472,5810.539292,321.987611,10 COLOUR SPACEBOY PEN
0,2124.493520,2072,207.2,59.052765,41.516336,597.439849,1456.694684,817.738499,2012.947235,111.546285,12 COLOURED PARTY BALLOONS
0,340.411124,332,33.2,9.462123,17.937116,258.123635,629.364324,353.303574,322.537877,17.873246,12 DAISY PEGS IN WOOD BOX
0,141.496190,138,13.8,3.933051,65.008088,935.497344,2280.956000,1280.450568,134.066949,7.429241,12 EGG HOUSE PAINTED WOOD
0,41.013388,40,4.0,1.140015,2.371231,34.123139,83.200000,46.705630,38.859985,2.153403,12 HANGING EGGS HAND PAINTED
...,...,...,...,...,...,...,...,...,...,...,...
0,NaN,800,80.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,wrongly coded 20713
0,NaN,1000,100.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,wrongly coded 23343
0,NaN,48,4.8,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,wrongly marked
0,NaN,200,20.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,wrongly marked 23343


In [18]:
empty_data.iloc[1,:]

quantity                          2124.493520
demand                                   2072
sd                                 207.200000
unitshort                           59.052765
shortagecost                        41.516336
cost                               597.439849
revenue                           1456.694684
profit                             817.738499
soldatfullprice                   2012.947235
sold_discount                      111.546285
decription         12 COLOURED PARTY BALLOONS
Name: 0, dtype: object